# 06 | ChatGPT 来了，让我们快速做个 AI 应用

In [2]:

import openai
import os

openai.api_key = os.environ.get("OPENAI_API_KEY")

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append({"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})

        if len(self.messages) > self.num_of_round*2 + 1:
            # Remove the first round conversation left.
            del self.messages[1:3]
        return message
    

prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""
conv1 = Conversation(prompt, 2)
question1 = "你是谁？"
print("User : %s" % question1)
print("Assistant : %s\n" % conv1.ask(question1))

question2 = "请问鱼香肉丝怎么做？"
print("User : %s" % question2)
print("Assistant : %s\n" % conv1.ask(question2))

question3 = "那蚝油牛肉呢？"
print("User : %s" % question3)
print("Assistant : %s\n" % conv1.ask(question3))

question4 = "我问你的第一个问题是什么？"
print("User : %s" % question4)
print("Assistant : %s\n" % conv1.ask(question4))

question5 = "我问你的第一个问题是什么？"
print("User : %s" % question5)
print("Assistant : %s\n" % conv1.ask(question5))


User : 你是谁？
Assistant : 我是一个AI语言模型，可以用中文回答你的做菜问题。

User : 请问鱼香肉丝怎么做？
Assistant : 鱼香肉丝的做法：
1. 猪肉切丝，加少许盐、料酒、淀粉拌匀，静置10分钟。
2. 胡萝卜、木耳、青红椒切丝备用。
3. 葱姜蒜切末，热锅凉油，爆香后加入肉丝煸炒至变色。
4. 加入胡萝卜、木耳、青红椒继续煸炒，加入适量盐、糖、酱油、醋、料酒、鸡精调味。
5. 倒入适量水淀粉勾芡，淋上香油即可。

User : 那蚝油牛肉呢？
Assistant : 蚝油牛肉的做法：
1. 牛肉切薄片，加少许盐、生抽、淀粉拌匀，静置10分钟。
2. 葱姜蒜切末，青红椒切丝备用。
3. 热锅凉油，爆香葱姜蒜末，加入牛肉煸炒至变色。
4. 加入青红椒丝继续煸炒，加入适量盐、糖、料酒、蚝油调味。
5. 倒入适量水淀粉勾芡，淋上香油即可。

User : 我问你的第一个问题是什么？
Assistant : 您的第一个问题是：“请问鱼香肉丝怎么做？”

User : 我问你的第一个问题是什么？
Assistant : 您的第一个问题是：“请问鱼香肉丝怎么做？”



## 通过 API 计算 Token 数量

In [2]:
import openai
import os

openai.api_key = os.environ.get("OPENAI_API_KEY")


class Conversation2:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append( {"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        num_of_tokens = response['usage']['total_tokens']
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message, num_of_tokens

prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""

question1 = "你是谁？"
question2 = "请问鱼香肉丝怎么做？"
question3 = "那蚝油牛肉呢？"
question4 = "我问你的第一个问题是什么？"
question5 = "我问你的第一个问题是什么？"


conv2 = Conversation2(prompt, 3)
questions = [question1, question2, question3, question4, question5]
for question in questions:
    answer, num_of_tokens = conv2.ask(question)
    print("询问 {%s} 消耗的token数量是 : %d" % (question, num_of_tokens))


询问 {你是谁？} 消耗的token数量是 : 105
询问 {请问鱼香肉丝怎么做？} 消耗的token数量是 : 344
询问 {那蚝油牛肉呢？} 消耗的token数量是 : 581
询问 {我问你的第一个问题是什么？} 消耗的token数量是 : 617
询问 {我问你的第一个问题是什么？} 消耗的token数量是 : 628


### 使用 tiktoken 库提前计算

安装依赖
```
conda activate py310
pip3 install --upgrade tiktoken
```


In [3]:

import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

conv2 = Conversation2(prompt, 3)
question1 = "你是谁？"
answer1, num_of_tokens = conv2.ask(question1)
print("总共消耗的token数量是 : %d" % (num_of_tokens))

prompt_count = len(encoding.encode(prompt))
question1_count = len(encoding.encode(question1))
answer1_count = len(encoding.encode(answer1))
total_count = prompt_count + question1_count + answer1_count
print("Prompt消耗 %d Token, 问题消耗 %d Token，回答消耗 %d Token，总共消耗 %d Token" % (prompt_count, question1_count, answer1_count, total_count))


总共消耗的token数量是 : 103
Prompt消耗 65 Token, 问题消耗 5 Token，回答消耗 20 Token，总共消耗 90 Token


## 使用 Gradio 快速搭建一个聊天界面

安装依赖
```
conda install -c conda-forge gradio
```

In [4]:

import gradio as gr
import openai
import os

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append({"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})

        if len(self.messages) > self.num_of_round*2 + 1:
            # Remove the first round conversation left.
            del self.messages[1:3]
        return message


prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""

conv = Conversation(prompt, 10)

def answer(question, history=[]):
    history.append(question)
    response = conv.ask(question)
    history.append(response)
    responses = [(u,b) for u,b in zip(history[::2], history[1::2])]
    return responses, history

with gr.Blocks(css="#chatbot{height:300px} .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)

    txt.submit(answer, [txt, state], [chatbot, state])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
